#Overview & motivation

All of our team members enjoy movies. In addition to enjoying movies, we also enjoy working with API’s and somewhat structured data sets. Therefore, determining what makes a movie successful using the data available in the Internet Movie Database (IMDB) and Wikipedia seemed like a natural choice.

In [6]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io

###Related Work

#Initial Questions

#Data

In [10]:
# Load AAdict (dict of Oscar nominated movies)
AAdict = pickle.load(open('AAdict','rb'))
# Load movies (dict of all movies)
movies = pickle.load(io.open('moviestemp.p','rb'))

#Exploratory Data Analysis

#Final Analysis

#Presentation